In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Loading training data**

In [ ]:
df = pd.read_csv("../input/tabular-playground-series-mar-2022/train.csv")

In [ ]:
df.head()

In [ ]:
df.info()

**Dtype of time and direction is object, so we need to convert them to numbers.
Let's start with direction.**

In [ ]:
df['direction'].value_counts()

In [ ]:
cat_df = df[['direction']]
df = df.drop(['direction'],axis =1)

**Using ordinal encoding to encode direction**

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
cat_df = ordinal_encoder.fit_transform(cat_df)
cat_df = pd.DataFrame(cat_df,columns=['direction'])

In [ ]:
df = pd.concat([df,cat_df], axis=1)

In [ ]:
df.head()

**Now we can create seperate columns for day, month, year, hour, minute and second**

In [ ]:
df['time'] = pd.to_datetime(df['time'])

In [ ]:
df['day'] = df['time'].dt.day
df['month'] = df['time'].dt.month
df['year'] = df['time'].dt.year
df['hour'] = df['time'].dt.hour
df['minute'] = df['time'].dt.minute
df['second'] = df['time'].dt.second

In [ ]:
df.head(100)

In [ ]:
df = df.drop('time', axis =1)

In [ ]:
df.info()

**Now every column has either int or float Dtype, so we can continue with any algorithm.**

In [ ]:
y_train = df['congestion']
x_train = df.drop(['congestion'],axis=1)

**Training Linear Regression Model**

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(x_train,y_train)

**Preparing test set**

In [ ]:
x_test = pd.read_csv("../input/tabular-playground-series-mar-2022/test.csv")
x_test.head()

In [ ]:
cat_test = x_test[['direction']]
x_test = x_test.drop(['direction'],axis =1)
cat_test = ordinal_encoder.transform(cat_test)
cat_test = pd.DataFrame(cat_test,columns=['direction'])
x_test = pd.concat([x_test,cat_test], axis=1)

In [ ]:
x_test['time'] = pd.to_datetime(x_test['time'])
x_test['day'] = x_test['time'].dt.day
x_test['month'] = x_test['time'].dt.month
x_test['year'] = x_test['time'].dt.year
x_test['hour'] = x_test['time'].dt.hour
x_test['minute'] = x_test['time'].dt.minute
x_test['second'] = x_test['time'].dt.second
x_test = x_test.drop(['time'],axis=1)

In [ ]:
x_test.info()

**Getting predictions from our trained model**

In [ ]:
predictions = reg.predict(x_test)

**Saving result in the desired format**

In [ ]:
output = pd.DataFrame({'row_id': x_test.row_id, 'congestion': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")